Kort forklaring om LFSR, hva det er og hva det brukes til? Kontekst. Hva er input og output?

In [3]:
import math
from functools import reduce
from operator import xor
from itertools import islice

Forklare hjelpemetoder og check-metoder

In [17]:
def check_input(polynomaial, state):
    
    if not (type(polynomaial) is int or type(polynomaial) is str or type(polynomaial) is list or type(polynomaial) is bin):
        raise TypeError("[!] Polynomaial has to be either an int, string, list or binary")
    if not (type(state) is int or type(state) is str or type(state) is bin or type(state) is None): 
        raise TypeError("[!] State has to be either an int, string or binary")
    
    #TODO sjekke at det kun er 0 og 1 i state
    """
    else: 
        for digit in str(int(float(state))): 
            if digit != '1' and digit != '0':
                raise TypeError("[!] State has to consist of only 0 and 1")
    """
    print("Inputs are ok!")
    

Description of the implementation: WHat have i done through the code, reason my choices, show my "path" and explain why i did it like this. 


In [43]:
class LFSR(object): 

    def __init__(self, poly, state=None):

        check_input(poly, state)

        print('Poly: ', poly)
        print('State: ', state)

        self.length = max(poly) 
        self.output = None
        self.feedback = 0
        self.state = state

        print(type(self.state))
        
        #convert poly to list of booleans
        self.poly = [False for i in range(max(poly))]
        for i in range(len(self.poly)):
            if i+1 in poly: #+1 fordi inputen poly ikke er 0-indeksert men det er lista
                self.poly[i] = True
        self.poly.reverse() #reverse for enkelhets skyld fordi P3 XORes med S0

        #convert state to list of booleans
        if state is None: 
          self.state = [True for i in range(max(poly))] #default state is 111...
        else: 
          if type(state) is int or type(state) is bin:
            self.state = str(state)
          self.state = [True if digit == '1' else False for digit in self.state]
    
    def __iter__(self): 
        return self

    def __next__(self): 
        anded = []
        for i in range(len(self.poly)):
          anded.append(self.state[i] and self.poly[i])
        self.feedback = reduce(xor, anded)
        self.output = self.state[0]
        self.state.pop(0)
        self.state.append(self.feedback)
        return self.output

    def run_steps(self, N=1): 
        list_of_bool = []
        for i in range(N):
          list_of_bool.append(self.__next__())
        return list_of_bool 
    
    def cycle(self, state=None): 
        N = (2**self.length)-1
        list_of_bool = self.run_steps(N)
        return list_of_bool

    def __str__(self, list):
      for b in islice(list, 8):
        print(f'{b:d}', end='')

The cyphertext and the plain text. A proof/demonstration of the code

In [5]:
lfsr = LFSR([3, 1], state=)
print(lfsr.poly)
#bitstream = lfsr.cycle()
#print(bitstream)

SyntaxError: invalid syntax (<ipython-input-5-28fd337efec4>, line 1)

In [13]:
lfsr = LFSR([3,1,0], state="111")
for b in islice(lfsr, 8):
    print(f'{b:d}', end='')

11101001

Conclusion: Just a few lines. 
What was the problem.
How you faced it.
How you solved it.
How you proved that you have solved it.

Drøfte litt? hva var utfordrende osv?

In [44]:
import numpy as np
from functools import reduce
from operator import xor

def berlekamp_massey(bits):
  N = len(bits)
  P = [0 for i in range(N)]
  P[N-1] = 1
  m = 0
  Q = [0 for i in range(N)]
  Q[N-1] = 1 
  r = 1

  """"
  print('====Attributes====')
  print('N: ', N)
  print('P: ', P)
  print('m: ', m)
  print('Q: ', Q)
  print('r: ', r)

  print('=====start======')
  """

  for t in range(N):
    anded = []
    x_r = [0 for i in range(N)]

    #check negative indexing
    for j in range(m+1):

      P_inverted = P[::-1]
      anded.append(P_inverted[j] and bits[t-j])
      d = reduce(xor, np.array(anded))
    
    """
    print('bt:', bit[t])
    print('P:', P)
    print('m:', m)
    print('Q:', Q)
    print('r:', r)
    """

    if d == 1: 
      if 2 * m <= t:
        R = P
        x_r[N-r-1] = 1
        Qx_r = bin_to_int(Q) * bin_to_int(x_r)
        P_int = bin_to_int(P)
        P_int = np.bitwise_xor(P_int, Qx_r)

        P = int_to_bin(P_int, N)
        Q = R
        m = t + 1 - m
        r = 0
      else: 
        x_r[N-r-1] = 1
        Qx_r = bin_to_int(Q) * bin_to_int(x_r)
        P_int = bin_to_int(P)
        P_int = np.bitwise_xor(P_int, Qx_r)

        P = int_to_bin(P_int, N)

    r = r + 1

  pol = P[::-1][:m+1]
  return_poly = [i for i in range(1, len(pol)) if pol[i]==1]
  return_poly.sort(reverse=True)
  return return_poly


def bin_to_int(fist_bin):
  return int("".join(str(x) for x in fist_bin), 2)

def int_to_bin(int_numb, N):
  binary = [1 if digit == '1' else 0 for digit in bin(int_numb)[2:]]
  return [0 for i in range(N - len(binary))] + binary


bit = [1, 0, 1, 0, 0, 1, 1, 1]
poly = berlekamp_massey(bit)
print('Final poly: ', poly)

Final poly:  [3, 1]


In [25]:
from itertools import islice
import random

class PseudoRandomByteGenerator(object):
    ''' class docstring '''

    def __init__(self, state=None, **kwargs):

        """
        self.poly = kwargs['poly']
        self.seed = seed
        """

        """
        if self.seed == None: 
            self.seed = '0' * max(self.poly)
        while len(self.seed) < max(self.poly): 
            self.seed = '0' + self.seed
        """
        
        self.bits = []
        self.lfsr_inst = LFSR(poly=kwargs['poly'], state=state)

    def __iter__(self):
        return self

    def bit_to_bytes(self, s):
        return int(s, 2).to_bytes(len(s) // 8, byteorder='big')
    
    def __next__(self):
        self.bits = self.lfsr_inst.run_steps(N=8)   #get a list of 8 bits represented as true and false
        self.bits = ''.join(['1' if b==True else '0' for b in self.bits])   #convert the list of bits to a string of 0 and 1
        byte = int(self.bits, 2)
        return byte

In [50]:
prbg = PseudoRandomByteGenerator(poly=[3, 1]) # iterator
digits = [byte for byte in islice(prbg, 10)]
print(bytes(digits))

b'\xe9\xd3\xa7N\x9d:t\xe9\xd3\xa7'


In [45]:
class StreamCipher():
    ''' class docstring '''
    def __init__(self, key, **kwargs):
        
        self.byte_gen = PseudoRandomByteGenerator(state=key, **kwargs)
    
    def encrypt(self, plaintext):
        N = len(plaintext)
        digits = [byte for byte in islice(self.byte_gen, N)]
        encrypted = [a ^ b for (a,b) in zip(bytes(digits), plaintext)]
        ciphertext = bytes(encrypted)
        return ciphertext
    
    def decrypt(self, ciphertext):
        N = len(ciphertext)
        digits = [byte for byte in islice(self.byte_gen, N)]
        decrypted = [a ^ b for (a,b) in zip(bytes(digits), ciphertext)]
        plaintext = bytes(decrypted)
        return plaintext

In [66]:
message = 'hello world!'
key = 0x0123456789ABCDEF
alice = StreamCipher(key, poly=[7, 1, 0])
plaintextA = message.encode('utf-8')
ciphertext = alice.encrypt(plaintextA)
bob = StreamCipher(key, poly=[7, 1, 0])
plaintextB = bob.encrypt(ciphertext)

print(plaintextA) # -> b'hello world!'
print(ciphertext) # -> b'\x9f\x03\xbcf\xfa\xdb`\xf6\x17\xce7\x88'
print(plaintextB) # -> b'hello world!'

Inputs are ok!
Poly:  [7, 1, 0]
State:  81985529216486895
<class 'int'>
Inputs are ok!
Poly:  [7, 1, 0]
State:  81985529216486895
<class 'int'>
b'hello world!'
b'(%DI\x7f\x90\xe9$\xf49m\xaa'
b'hello world!'


In [69]:
from bitarray import bitarray
import binascii 
import codecs

def access_bit(data, num):
    base = int(num // 8)
    shift = int(num % 8)
    return (data[base] & (1<<shift)) >> shift

with open("proj1_ciphertext.bin", "rb") as ciphertext:
    cipher_data = ciphertext.read()
    print(cipher_data)
    cipher_bitstream = [access_bit(cipher_data,i) for i in range(len(cipher_data)*8)]
    #print(cipher_bitstream)
    #bitstream = ''.join(bits)

with open("proj1_known-plaintext.txt", 'r') as plaintext:
    ba = bitarray()
    plain_data = plaintext.read()
    plain_bitstream = bin(int(binascii.hexlify(bytearray(plain_data, 'utf-8')),16))
    plain_bits = [int(plain_bitstream[i]) for i in range(2, len(plain_bitstream))]
    #print(plain_bits)

    """
    bytearrayy = bytearray(data, 'utf-8')
    print(bytearrayy)

    plain_bits = [format(x, 'b') for x in bytearray(data, 'utf-8')]
    print(plain_bits)
    plain_bitstream = [int(i) for i in plain_bits]
    print(plain_bitstream)
"""
    
def xor_texts():   
    key = []
    print(type(plain_bitstream[0]))
    print(type(cipher_bitstream[0]))
    for i in range(len(plain_bits)): 
        #print(xor(plain_bits[i], cipher_bitstream[i]))
        key.append(xor(plain_bits[i], cipher_bitstream[i]))
    return_key = [int(i) for i in key]
    #print(return_key)
    return return_key

keyy = xor_texts()  
poly = berlekamp_massey(keyy)
ke = [str(i) for i in keyy]
key = ''.join(ke)
print('Key: ', key)
bob = StreamCipher(key=key, poly=poly)

#plaintextA = plain_data.encode('utf-8')
#ciphertext = streamcipher.encrypt(plaintextA)
#a = streamcipher.encrypt(ciphertext)

"""
alice = StreamCipher(key, poly=poly)
plaintextA = plain_data.encode('utf-8')
ciphertext = alice.encrypt(plaintextA)
bob = StreamCipher(key, poly=poly)
plaintextB = bob.encrypt(ciphertext)

print('Plaintext', plaintextA)
print('Ciphertext: ', ciphertext)
print('Plain text: ', plaintextB)
"""

plaintextB= streamcipher.encrypt(cipher_data)
#answer = bytearray.fromhex(decoded).decode()
print(plaintextB)


xb8X-\x04u\xaaPO\xff\xe4\xdcd\x9b\x05\x8d\xe2Cw!\xf4\x87\x8b\x95/I\x8b\xfd5\xc7\x94\x1b\xdf\x85\xfa\x8c\xdc\xea\x97\xf1\x7f\xack\xb6c\x95\xfa\xb14,\x80\xae\x89\x1b\x04\x9fI0\x95\xceL\xb6\x1b\xde\x16\x1b+v\xbd\xb7\x15\xbe\xa8\x87@Jp\x1f#\x8f\x83}*r\r\xdb\xce\x9c6e\x1b\x93\xf5KU\xaf\xcc\x1aH\\\x82@\\\x03\xa7\x9ba\xdcnQ\xd6\xca\xf1\xc7M\xdc^9\xfc\x1d+i\xdc\x18\xa0\\\xf0\x88\x7f\xf5\xf2c\x13\x10~V\x0e\x10\xec\xbc\xf3\xa0\xf2\xb9\x077\xfd0\xb9ON\xd3d\xe1\xb6T\x8a\x7f\xef\x9b\xce\xa5\xac*Vq\xfdm\xd0\xd7!\x1e\xffo*\x12A\xcd,\x8a-7Z\xbea)\x86p\x17\xad\x05\x9e;\x8e\x01\xf3x,\xae\xf8x3\xc3R\x85\x87\xddJ\xe9pK"\xc8\xab\xff\n\xe2\xc9\x04f\xd6\xa8\xc0\xc2\x8b0\xdd\xb7b\xe2C\'\x0cB0x\x98:4zmM\xe0X\xbc\xab\xa8\xd85\xa3\xd7\xfe|\x82\xad\x8a\xa7P;\x14s\xa6\x11E\xe4\xf2\xcd-\x8f@\xc1\xe3\x06`&\xb5\x80\x96\x95>L\x8e\xe2c\xed\x9e\x19\x93\x8e\xbc\xcd\x90\xee\x80\xfby\xa2%\xb1&\x9f\xfb\xbb"3\xff\xd4\xea\x03\x18\x87E0\x9d\x8aD\xb6\x0b\x96\x0b\x1d/"\xf8\xa9\x11\xa8\xa8\x8bJ\x1e#\x1c/\x8e\xcd)5 \n\xdd\xca\x98{